# Notebook

# Step 4: Interactive Visual Analytics with Folium

In this notebook, we'll create an interactive geographical map to visualize the SpaceX launch sites. This will help us understand the location of each site and we will add custom markers to show their success rates.

**Key Tasks:**
1. Load launch site coordinate data from our raw API file.
2. Load our cleaned launch data to calculate success rates per site.
3. Create a base map using Folium.
4. Add markers for each launch site, colored by their success rate.
5. Save the interactive map as an HTML file.```




## 4.1: Setup and Imports

In [1]:
import pandas as pd
import yaml
import os
import folium
from folium.plugins import MarkerCluster

print(f"Folium version: {folium.__version__}")

Folium version: 0.20.0


## 4.2: Load Data

We need two datasets for this task:
1. The raw `spacex_api_data.json` file, which contains the names and coordinates of the launchpads.
2. Our `cleaned_launches.csv` file, which we'll use to calculate the success rate for each launch site.

In [5]:
import pandas as pd
import yaml
import os
import requests # We need this to make the second API call

# --- START OF THE FINAL, CORRECT FIX ---
# 1. Fetch the Launchpad Details from the correct API endpoint
print("Fetching launchpad details from API...")
try:
    launchpads_url = "https://api.spacexdata.com/v4/launchpads"
    response = requests.get(launchpads_url)
    response.raise_for_status()
    launchpads_data = response.json()
    
    # Create a simple lookup dictionary: {launchpad_id: {name: ..., lat: ..., lon: ...}}
    launchpad_lookup = {
        pad['id']: {
            'name': pad.get('name'),
            'lat': pad.get('latitude'),
            'lon': pad.get('longitude')
        } for pad in launchpads_data
    }
    print("Successfully created launchpad lookup map.")
except Exception as e:
    print(f"Could not fetch launchpad details: {e}")
    launchpad_lookup = {}

# --- Load our Cleaned Launch Data ---
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)
INTERIM_DATA_PATH = os.path.join('..', config['data_paths']['interim'])
WRANGLED_DATA_FILE = os.path.join(INTERIM_DATA_PATH, config['data_files']['wrangled_data'])
launches_df = pd.read_csv(WRANGLED_DATA_FILE)

# --- Prepare Data for the Map ---
# 1. Calculate success rates from our clean data. The 'LaunchSite' column contains the ID.
site_success_rates = launches_df.groupby('LaunchSite')['class'].mean()

# 2. Build our final sites_df by iterating through the success rates
site_data = []
for site_id, success_rate in site_success_rates.items():
    # Look up the details for this ID in our new map
    details = launchpad_lookup.get(site_id)
    if details and details['lat'] is not None and details['lon'] is not None:
        site_data.append({
            'LaunchSite': details['name'],
            'Latitude': details['lat'],
            'Longitude': details['lon'],
            'class': success_rate
        })

sites_df = pd.DataFrame(site_data)
# --- END OF THE FINAL, CORRECT FIX ---

print("\nLaunch Sites data prepared for mapping:")
display(sites_df)

Fetching launchpad details from API...
Successfully created launchpad lookup map.

Launch Sites data prepared for mapping:


,LaunchSite,Latitude,Longitude,class
0,CCSFS SLC 40,28.561857,-80.577366,0.642857
1,VAFB SLC 4E,34.632093,-120.610829,0.766667
2,KSC LC 39A,28.608058,-80.603956,0.827586
3,Kwajalein Atoll,9.047721,167.743129,0.000000


## 4.3: Create and Display the Interactive Map

Now that we have a clean dataframe with site names, coordinates, and success rates, we can create the map and add a custom marker for each site.

In [6]:
# Create a base map centered on the USA
spacex_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Create a MarkerCluster object for a cleaner look when sites are close together
marker_cluster = MarkerCluster().add_to(spacex_map)

# Iterate through our prepared sites_df and add a colored marker for each launch site
for index, row in sites_df.iterrows():
    site_name = row['LaunchSite']
    latitude = row['Latitude']
    longitude = row['Longitude']
    success_rate = row['class']
    
    # Define a function to determine the marker color based on success rate
    def get_color(rate):
        if rate >= 0.75:
            return 'green'      # High success rate
        elif rate >= 0.5:
            return 'orange'     # Medium success rate
        else:
            return 'red'        # Low success rate
    
    # Create the popup text that will appear when you click on a marker
    popup_text = f"<strong>{site_name}</strong><br>Success Rate: {success_rate:.2%}"
    
    # Create the marker with a custom rocket icon
    marker = folium.Marker(
        location=[latitude, longitude],
        popup=popup_text,
        icon=folium.Icon(color=get_color(success_rate), icon='rocket', prefix='fa')
    )
    
    # Add the marker to our cluster
    marker.add_to(marker_cluster)

# Display the final map directly in the notebook output
spacex_map

## 4.4: Save the Map

Finally, let's save this interactive map as a standalone HTML file. This file can be opened in any web browser or embedded in a report or website.

In [7]:
# The map can be saved to be used in the dashboard or report.
# We will save it in the 'docs' folder for this project.
map_path = os.path.join('..', 'docs', 'launch_site_map.html')
spacex_map.save(map_path)

print(f"Interactive map saved successfully to: {map_path}")

Interactive map saved successfully to: ..\docs\launch_site_map.html
